# Контекст

# PythonOperator

In [ ]:
from airflow import DAG
from airflow.operators.python_operator import PythonOperator
from datetime import datetime

########################################################################################

from airflow.operators.python import get_current_context

########################################################################################
with DAG('dag', schedule_interval='@daily', 
         
          start_date=datetime(2021, 1, 1),
          end_date=datetime(2021, 1, 10)) as dag:

  def my_func(hello):

    ########################################################################################    
    
    context = get_current_context()
    
    ########################################################################################
    ds = context['ds']
    task = context['task']
    print(hello)
    print(ds)
    print(task)
    print("Контекст", context) 
    

  python_task = PythonOperator(
    task_id='python_task', 
    python_callable=my_func,
    op_kwargs= {
      'hello': 'Hello World'
      }
    )

In [ ]:
from airflow import DAG 
from datetime import datetime 
from airflow.operators.python_operator import PythonOperator 

default_args = {"owner": "airflow", "start_date": datetime(2018, 10, 1)} 
dag = DAG(dag_id="dag", default_args=default_args, schedule_interva="@daily") 

# Функция использующая контекст 
def _print_exec_date(**context): 
    print("Контекст", context) 
    
print_exec_date = PythonOperator( 
    task_id="print_exec_date", 
    python_callable=_print_exec_date, 
    dag=dag )

# BashOperator

In [ ]:
from airflow import DAG 
from datetime import datetime 
from airflow.operators.bash_operator import BashOperator 

default_args = {"owner": "airflow", "start_date": datetime(2018, 10, 1)} 
dag = DAG(dag_id="dag", default_args=default_args, schedule_interval="@daily") 

bash_task = BashOperator( 
    task_id="bash_task", 
    bash_command='echo "Context: \'$message\'"', 
    env={'message': '{{ execution_date }}'}, # Используется jinja выражение
    dag=dag )

In [ ]:
Jinja
Если вы не знакомы с Jinja, то это просто текстовый шаблонизатор, он позволяет писать код через текстовые выражения и запускать как обычный python скрипт.

Например 

t = Template("Hello {{ something }}!") # Создадим шаблон
t.render(something="World") # Отрендерим результат
>> u'Hello World!' # Результат работы

Или вот так

from jinja2 import Template

# Выведем список чисел через в цикле который реализуем через Jinja
t = Template("My favorite numbers: {% for n in range(1,10) %}{{n}} " "{% endfor %}")
t.render()
>> My favorite numbers: 1 2 3 4 5 6 7 8 9 

Jinja создана в первую очередь для веб разработки где удобно генерировать HTML теги. Но в Airflow его используют для упрощения кода и доступа к различным элементам. Например для доступа к элементам контекста или Xcom (про это позже). Пример выражений которые можно использовать в операторах.

Xcom хранилища

SQLite - 2 gb
PostreSQL - 1gb
MySQL - 64 kb 

In [ ]:
from airflow import DAG
from datetime import timedelta
from airflow.utils.dates import days_ago
from airflow.operators.bash import BashOperator

dag = DAG('dag',schedule_interval=timedelta(days=1), start_date=days_ago(1))

downloading_data = BashOperator(
    task_id='downloading_data',
    bash_command='echo "Hello, I am a value!"',
    do_xcom_push=True, # Результат работы будет отправлен в Xcom
    dag=dag
)

fetching_data = BashOperator(
    task_id='fetching_data',
    # Через Jinja также можно отправлять xcom
    bash_command="echo 'XCom fetched: {{ ti.xcom_pull(task_ids=[\'downloading_data\']) }}'",
    do_xcom_push=False, # Результат работы НЕ будет отправлен в Xcom
    dag=dag
)

downloading_data >> fetching_data





Результатом выполнения первого таска является запись в БД
А результатом получения данных является запись в логе

Важно

Я ещё раз уточню что Xcom в самом простом варианте использую только для передачи метаданных, и только если подключить S3 хранилище можно использовать его как способ обмена между тасками.

In [ ]:
import airflow
from airflow import DAG
from datetime import datetime
from airflow.utils.dates import days_ago
from airflow.operators.python_operator import PythonOperator


args = {'owner':'airflow', "start_date": days_ago(1)}
dag = DAG(dag_id="dag", default_args=args, schedule_interval="@once")

value_2 = {'a':'b'}

def push_func(**kwargs):
    kwargs['ti'].xcom_push(key='key', value=value_2)
    kwargs['ti'].xcom_push(key='key2', value=str(kwargs['execution_date']))
    
def pull_func(**kwargs):
    kwargs['ti'].xcom_pull(key='key', task_ids='push_func')
    kwargs['ti'].xcom_pull(key='key2', task_ids='push_func')
    

push_1 = PythonOperator( 
    task_id="push_func", 
    python_callable=push_func, 
    dag=dag )

pull_1 = PythonOperator( 
    task_id="pull_func", 
    python_callable=pull_func, 
    dag=dag )
    
push_1 >> pull_1